In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import time
import random
from tqdm import tqdm_notebook

In [3]:
import re
import pandas as pd

### [Naver news에서 Title, Publisher, Date, Url, Text 가져오기]

In [37]:
def get_urls(url):
    
    titles, urls, pubs = [], [], []
    dates, texts = [], []
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
    lis = soup.find_all('li', attrs={'id':re.compile(r'sp_nws')})
    
    for li in lis:
# news title, publisher, url 가져오기
        title = li.find('a', attrs={'class':re.compile(r'_sp_each_title')}).get('title').strip()       
        if li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).text:
            news_url = li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).get('href').strip()        
        pub = li.find('span', attrs={'class':'_sp_each_source'}).text.strip()
        titles.append(title)
        urls.append(news_url)
        pubs.append(pub)
# 가져온 url로 news date, text 가져오기      
        if 'news.naver.com/main' in news_url:
            r = requests.get(news_url)
            soup = BeautifulSoup(r.text, 'lxml')
            try:
                text = soup.find('div', attrs = {'id': 'articleBody'}).text
                texts.append(text.split('이 기사는')[0].split('}')[1].strip())
            except:
                text = soup.find('div','end_body_wrp').text
                texts.append(text.split('▶')[0].strip())
            dates.append(soup.find('span', class_='t11').text[:12])
        else:
            texts.append(None)
            dates.append(None)
        
        #print(title, naver_url, pub)
        
    return titles, urls, pubs, dates, texts

- SKT 누구 출시일- 2016.09<br>
- KT 기가지니 출시일- 2017.01<br>
- 네이버 클로바 출시일- 2017.10<br>
- 카카오 미니 출시일- 2017.11<br>

In [141]:
date_pd = pd.date_range('2017-10-01', '2019-05-31').strftime("%Y%m%d")

In [142]:
q = '"카카오%20미니"%20스피커'

total_titles, total_urls, total_pubs, total_dates, total_texts = [], [], [], [], []

for date in tqdm_notebook(date_pd):
    for page in range(1,100):
        try: 
            url = 'https://search.naver.com/search.naver?&where=news&query='+q+'&sm=tab_pge&sort=0&docid=&nso=so:r,p:from'+str(date)+'to'+str(date)+',a:all&start='+str(page)

            title, url, pub, date, text = get_urls(url)
            if url[0] in total_urls:# 페이지 끝나면 멈춤
                break

            total_titles += title
            total_urls += url
            total_pubs += pub
            total_dates += date
            total_texts += text
            
        except:
            break

In [143]:
len(total_titles), len(total_pubs), len(total_urls), len(total_dates), len(total_texts)

(1842, 1842, 1842, 1842, 1842)

In [147]:
naver_news = pd.DataFrame(data = {'title': total_titles, 'date': total_dates, 'publisher': total_pubs, 'url': total_urls, 'text': total_texts})
naver_news.head()

,title,date,publisher,url,text
0,카카오 미니 vs 네이버 웨이브가 고민인 당신에게,2017.10.02.,머니투데이,https://news.naver.com/main/read.nhn?mode=LSD&...,"[머니투데이 이해인 기자] [카카오 미니, 카톡 확인 및 전송 가능…네이버 웨이브,..."
1,네이버 ‘기술’... 카카오 ‘서비스’ 양대산맥 주목,None,뉴스웨이,http://www.newsway.co.kr/view.php?tp=1&ud=2017...,None
2,"카카오, AI 스피커 '카카오미니C' 고음질 업데이트",2019.05.08.,블로터언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...
3,스카이라이프 가입자도 AI 스피커 쓴다…대상 제품은 '구글 홈 미니',None,IT조선,http://it.chosun.com/site/data/html_dir/2019/0...,None
4,AI스피커 제조사 음질 경쟁…왜?,2019.05.12.,노컷뉴스언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...


In [148]:
len(naver_news[naver_news.date.notnull()]), len(naver_news[naver_news.text.notnull()])

(746, 746)

- sports news 가져오는 code
for i in naver_news[naver_news.date.isna()].url:
    if 'news.naver.com' in i:
        print(i)
for num, link in enumerate(naver_news[naver_news.date.isna()].url):
    if 'sports.news.naver.com' in link:
        r = requests.get(link)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
        naver_news.date = naver_news.date[num] = soup.find('div', class_='info').find('span').text.split('기사입력 ')[1][:11]
        naver_news.text[num] = soup.find('div', id= "newsEndContents").text.split('기자')[0]

In [149]:
naver_news.to_csv('kakaomini_speaker_news.csv',encoding='UTF-8')
pd.read_csv('kakaomini_speaker_news.csv', encoding='utf=8', index_col=0)

,title,date,publisher,url,text
0,카카오 미니 vs 네이버 웨이브가 고민인 당신에게,2017.10.02.,머니투데이,https://news.naver.com/main/read.nhn?mode=LSD&...,"[머니투데이 이해인 기자] [카카오 미니, 카톡 확인 및 전송 가능…네이버 웨이브,..."
1,네이버 ‘기술’... 카카오 ‘서비스’ 양대산맥 주목,NaN,뉴스웨이,http://www.newsway.co.kr/view.php?tp=1&ud=2017...,NaN
2,"카카오, AI 스피커 '카카오미니C' 고음질 업데이트",2019.05.08.,블로터언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...
3,스카이라이프 가입자도 AI 스피커 쓴다…대상 제품은 '구글 홈 미니',NaN,IT조선,http://it.chosun.com/site/data/html_dir/2019/0...,NaN
4,AI스피커 제조사 음질 경쟁…왜?,2019.05.12.,노컷뉴스언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...
5,"커넥트온, 카카오 미니 스피커 통해 `서울 관광명소` 안내 서비스 개시",2019.06.04.,디지털타임스,https://news.naver.com/main/read.nhn?mode=LSD&...,주식회사 커넥트온이 오늘(4일) 서울 관광명소를 안내하는 카카오 보이스봇 플러그인을...
6,"세계 AI스피커 시장, 美·中 2강구도 굳어지나",2019.06.02.,파이낸셜뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,인공지능(AI) 스피커 시장이 미국과 중국 IT업체의 2강구도가 될 조짐을 보이고 ...
7,삼성도 곧 참전…치열한 ‘AI 스피커’ 거실쟁탈전,2019.04.26.,ZDNet Korea언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,아마존과 구글이 이끌어…국내에선 통신사 주도(지디넷코리아=권혜미 기자)스마트폰 다음...
8,"""한꺼번에 말해도 괜찮아""...인공지능 음성 분리 기술 나왔다",NaN,이코노믹리뷰,http://www.econovill.com/news/articleView.html...,NaN
9,"카카오, AI스피커 시각장애인에 기증",NaN,미디어SR,http://www.mediasr.co.kr/news/articleView.html...,NaN
